In [3]:
import numpy as np
import pandas as pd
import plotly
#import iFeatureOmegaCLI
import Bio
import modlamp

In [9]:
protien = pd.read_csv("./sequences_training.txt", header=None)
protien.columns = ["protien_seq", "class"]
protien.head()

,protien_seq,class
0,MLKQVEIFTDGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...,nonDRNA
1,MEQKKMKYLENLVGKTPMLELIFDYKGEERRIFVKNESYNLTGSIK...,nonDRNA
2,MTILFQLALAALVILSFVMVIGVPVAYASPQDWDRSKQLIFLGSGL...,nonDRNA
3,MSKIERISAFLNDKEVDMTFITNPTTLNYLTGLAIDPHERIAGLMI...,nonDRNA
4,MSDQQQPPVYKIALGIEYDGSKYYGWQRQNEVRSVQEKLEKALSQV...,RNA


In [13]:
temp = protien.head()
#Bio.SeqUtils.ProtParam.ProteinAnalysis(temp)
Bio.__version__

'1.86'

Getting features based on Physicochemical and Structural Features using Bio

In [14]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis

In [18]:
temp2 = ProteinAnalysis(temp["protien_seq"][0])

In [30]:
#print(temp2.aromaticity())
print(temp2.molecular_weight())
print(temp2.isoelectric_point())
#print(temp2.charge_at_pH())
print(temp2.gravy())
print(temp2.secondary_structure_fraction())
#print(temp2.count_amino_acids())
print(temp2.amino_acids_percent)

17596.7878
8.42854824066162
-0.6309677419354839
(0.3419354838709678, 0.23870967741935487, 0.32903225806451614)
{'A': 9.03225806451613, 'C': 1.935483870967742, 'D': 4.516129032258065, 'E': 7.741935483870968, 'F': 1.2903225806451613, 'G': 9.03225806451613, 'H': 3.225806451612903, 'I': 4.516129032258065, 'K': 7.096774193548387, 'L': 7.741935483870968, 'M': 2.5806451612903225, 'N': 4.516129032258065, 'P': 3.225806451612903, 'Q': 5.161290322580645, 'R': 6.451612903225806, 'S': 2.5806451612903225, 'T': 6.451612903225806, 'V': 5.806451612903226, 'W': 3.870967741935484, 'Y': 3.225806451612903}


In [33]:
temp2.amino_acids_percent.keys()

dict_keys(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'])

Positive-to-Negative Residue Ratio:
$\frac{K+R+H}{D+E}$
This formula has the potential to lead to a division by 0, so I will be adding a very small number $\epsilon$:
$\frac{K+R+H}{D+E+\epsilon}$

In [37]:
temp3 = temp2.count_amino_acids()
print(temp3['K'] + temp3['R'] + temp3['H'] / temp3['D'] + temp3['E'])

33.714285714285715


Amino Acid Categories [Source](https://www.britannica.com/science/amino-acid/Standard-amino-acids)
- Type 1 Nonpolar:
  - A, G, I, L, M, F, P, W, V
- Type 2 Polar, Uncharged:
  - S, C, N, Q, T, Y
- Type 3 Acidic:
  - D, E
- Type 4 Basic:
  - K, R, H

*note: we could also try combining types 2, 3 and 4 into 1 category if needed*

Features I want to start with:
- molecular weight
- isoelectric point
- gravy score
- all 3 secondary structures
- all 20 amino acid percenages
- protien length
- positive-to-negative residue ratio
- all 4 amino acid category counts

In [40]:
# the mini dataframe I will be using to figure out how to extract my features on interest
temp

,protien_seq,class
0,MLKQVEIFTDGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...,nonDRNA
1,MEQKKMKYLENLVGKTPMLELIFDYKGEERRIFVKNESYNLTGSIK...,nonDRNA
2,MTILFQLALAALVILSFVMVIGVPVAYASPQDWDRSKQLIFLGSGL...,nonDRNA
3,MSKIERISAFLNDKEVDMTFITNPTTLNYLTGLAIDPHERIAGLMI...,nonDRNA
4,MSDQQQPPVYKIALGIEYDGSKYYGWQRQNEVRSVQEKLEKALSQV...,RNA


In [43]:
temp = temp.copy(deep=True)
temp["protien_an"] = temp["protien_seq"].apply(ProteinAnalysis)
temp

,protien_seq,class,protien_an
0,MLKQVEIFTDGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...,nonDRNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...
1,MEQKKMKYLENLVGKTPMLELIFDYKGEERRIFVKNESYNLTGSIK...,nonDRNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...
2,MTILFQLALAALVILSFVMVIGVPVAYASPQDWDRSKQLIFLGSGL...,nonDRNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...
3,MSKIERISAFLNDKEVDMTFITNPTTLNYLTGLAIDPHERIAGLMI...,nonDRNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...
4,MSDQQQPPVYKIALGIEYDGSKYYGWQRQNEVRSVQEKLEKALSQV...,RNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...


In [47]:
# repeat this process for the isoelectric point and gravy score functions
temp["mol_weighht"] = temp["protien_an"].apply(lambda x: x.molecular_weight())
temp

,protien_seq,class,protien_an,mol_weighht
0,MLKQVEIFTDGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...,nonDRNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...,17596.7878
1,MEQKKMKYLENLVGKTPMLELIFDYKGEERRIFVKNESYNLTGSIK...,nonDRNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...,37260.7955
2,MTILFQLALAALVILSFVMVIGVPVAYASPQDWDRSKQLIFLGSGL...,nonDRNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...,6764.1710
3,MSKIERISAFLNDKEVDMTFITNPTTLNYLTGLAIDPHERIAGLMI...,nonDRNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...,39968.9808
4,MSDQQQPPVYKIALGIEYDGSKYYGWQRQNEVRSVQEKLEKALSQV...,RNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...,30399.2510


In [59]:
temp_struc = pd.DataFrame(list(temp["protien_an"].apply(lambda x: x.secondary_structure_fraction())), columns=["Helix", "Turn", "Sheet"])
temp_struc

,Helix,Turn,Sheet
0,0.341935,0.238710,0.329032
1,0.366071,0.303571,0.366071
2,0.322581,0.209677,0.580645
3,0.334254,0.281768,0.356354
4,0.300000,0.248148,0.355556


In [51]:
temp2 = temp["protien_an"].apply(lambda x: x.count_amino_acids()).apply(pd.Series)
temp2

,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y
0,14,3,7,12,2,14,5,7,11,12,4,7,5,8,10,4,10,9,6,5
1,18,2,18,25,14,30,3,29,37,31,12,20,11,4,10,23,14,22,1,12
2,6,0,2,0,5,4,0,5,1,11,2,1,2,3,1,4,1,11,2,1
3,31,4,26,31,19,28,11,30,21,25,13,16,15,8,12,17,20,25,1,9
4,31,3,15,13,9,16,11,10,11,20,6,11,15,12,20,10,14,26,5,12


In [53]:
pd.concat([temp, temp2], axis=1)

,protien_seq,class,protien_an,mol_weighht,A,C,D,E,F,G,...,M,N,P,Q,R,S,T,V,W,Y
0,MLKQVEIFTDGSCLGNPGPGGYGAILRYRGREKTFSAGYTRTTNNR...,nonDRNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...,17596.7878,14,3,7,12,2,14,...,4,7,5,8,10,4,10,9,6,5
1,MEQKKMKYLENLVGKTPMLELIFDYKGEERRIFVKNESYNLTGSIK...,nonDRNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...,37260.7955,18,2,18,25,14,30,...,12,20,11,4,10,23,14,22,1,12
2,MTILFQLALAALVILSFVMVIGVPVAYASPQDWDRSKQLIFLGSGL...,nonDRNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...,6764.1710,6,0,2,0,5,4,...,2,1,2,3,1,4,1,11,2,1
3,MSKIERISAFLNDKEVDMTFITNPTTLNYLTGLAIDPHERIAGLMI...,nonDRNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...,39968.9808,31,4,26,31,19,28,...,13,16,15,8,12,17,20,25,1,9
4,MSDQQQPPVYKIALGIEYDGSKYYGWQRQNEVRSVQEKLEKALSQV...,RNA,<Bio.SeqUtils.ProtParam.ProteinAnalysis object...,30399.2510,31,3,15,13,9,16,...,6,11,15,12,20,10,14,26,5,12


In [60]:
temp2["D"] + temp2["E"]

0    19
1    43
2     2
3    57
4    28
dtype: int64